# 🧪 Importando el Mundo: Visualización y Conversión de Formatos 3D


**Computación visual**

## 1. Preparar el entorno

In [1]:
!pip install trimesh open3d numpy

In [2]:
import trimesh
import open3d as o3d
import numpy as np
import os
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Cargar y analizar los modelos

In [3]:
base_model_path = "12140_Skull_v3_L2.obj"
output_dir = "converted_models"
os.makedirs(output_dir, exist_ok=True)

model_files = {
    "OBJ": base_model_path,
    "STL": os.path.join(output_dir, "skull.stl"),
    "GLTF": os.path.join(output_dir, "skull.glb")
}


models = {} # Diccionario para almacenar los modelos cargados

# Cargar y analizar modelos
for format_name, file_path in model_files.items():
    try:
        # Cargar el modelo con trimesh
        model = trimesh.load(file_path, force='mesh')
        models[format_name] = model

        # Analizar propiedades
        vertices = model.vertices
        faces = model.faces
        normals = model.vertex_normals if hasattr(model, 'vertex_normals') else []
        num_vertices = len(vertices)
        num_faces = len(faces)
        num_normals = len(normals)

        # Detectar vértices duplicados
        unique_vertices, counts = np.unique(vertices, axis=0, return_counts=True)
        num_duplicates = len(vertices) - len(unique_vertices)

        # Imprimir resultados
        print(f"\nModelo: {format_name}")
        print(f"Número de vértices: {num_vertices}")
        print(f"Número de caras: {num_faces}")
        print(f"Número de normales: {num_normals}")
        print(f"Vértices duplicados: {num_duplicates}")

    except Exception as e:
        print(f"Error al cargar {format_name}: {e}")



Modelo: OBJ
Número de vértices: 42682
Número de caras: 80016
Número de normales: 42682
Vértices duplicados: 2620

Modelo: STL
Número de vértices: 40062
Número de caras: 80016
Número de normales: 40062
Vértices duplicados: 0

Modelo: GLTF
Número de vértices: 42682
Número de caras: 80016
Número de normales: 42682
Vértices duplicados: 2620


## Bonus: Automatizar comparación

In [4]:
# Bonus: Automatizar comparación
def compare_models(model_dict):
    comparison = {}
    for format_name, model in model_dict.items():
        vertices = model.vertices
        faces = model.faces
        unique_vertices = len(np.unique(vertices, axis=0))
        comparison[format_name] = {
            "vertices": len(vertices),
            "faces": len(faces),
            "unique_vertices": unique_vertices,
            "duplicates": len(vertices) - unique_vertices
        }

    # Imprimir comparación
    print("\nComparación de Modelos:")
    print("Formato | Vértices | Caras | Vértices Únicos | Duplicados")
    print("-" * 50)
    for format_name, stats in comparison.items():
        print(f"{format_name:7} | {stats['vertices']:8} | {stats['faces']:5} | {stats['unique_vertices']:13} | {stats['duplicates']:10}")

# Ejecutar comparación
compare_models(models)


Comparación de Modelos:
Formato | Vértices | Caras | Vértices Únicos | Duplicados
--------------------------------------------------
OBJ     |    42682 | 80016 |         40062 |       2620
STL     |    40062 | 80016 |         40062 |          0
GLTF    |    42682 | 80016 |         40062 |       2620


In [5]:
for format_name, model in models.items():
    vertices = np.asarray(model.vertices)
    triangles = np.asarray(model.faces)
    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(vertices)
    mesh.triangles = o3d.utility.Vector3iVector(triangles)
    mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([mesh], window_name=f"Modelo {format_name}")

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
